In [7]:
import numpy as np
from scipy.io import wavfile
from scipy.signal import iirpeak
from scipy.signal import iirnotch
from IPython.display import Audio 


### Implementacija prvog audio efekta - delay (kašnjenje)
Delay je gitarski audio efekat koji spada u grupu efekata zasnovanih na kašnjenju, i to je upravo i njegov prevod - kašnjenje. Realizuje se jednostavnim zakašnjenjem ulaznog signala i sabiranjem tog zakašnjelog signala sa originalnim signalom. Za potrebe našeg projektnog zadatka smješten je u Grupu 1 - Manje zahtjevni filtri.

In [2]:
def delay(x, D, mix):
    x = x / np.max(np.abs(x))
    x_delay = np.zeros(len(x))
    x_delay[D:] = x[:-D]

    y = (1-mix) * x + mix * x_delay
    return y

In [3]:
Fs, x = wavfile.read('acoustic.wav')
y = delay(x, int(0.2*Fs), 0.2)
Audio(y, rate=Fs)

### Implementacija drugog efekta - distortion (distorzija)
Jedan od najpoznatijih i najčešće korišćenih gitarskih efekata u rok muzici. Spada u grupu nelinearnih efekata. 

In [64]:
def distortion1(x, gain, mix):
    x = x / np.max(np.abs(x)) # normalizacija
    x_gain = gain * x
    x_dist = np.sign(x_gain) * (1 - np.exp(-np.sign(x_gain) * x_gain))
    x_dist *= (np.max(np.abs(x_gain))) / np.max(np.abs(x_dist))
    y = (1-mix) * x + mix * x_dist
    y *= np.max(np.abs(x_dist)) / np.max(np.abs(y))
    return y

In [65]:
def distortion2(x, gain, mix):
    x = x / np.max(np.abs(x))       # normalizacija
    x_gain = gain * x
    x_dist = np.sign(x_gain) * (1 - np.exp(-np.sign(x_gain) * x_gain))
    y = (1-mix) * x + mix * x_dist
    return y

In [66]:
Fs, x = wavfile.read('acoustic.wav')
y1 = distortion1(x, 10, 0.8)
y2 = distortion2(x, 10, 0.8)
Audio(y1, rate=Fs)

In [67]:
Audio(y2, rate=Fs)

### Implementacija trećeg efekta - wah-efekat. 
Ovaj efekat je dobio ime po tome, što kada se primjeni na neki audio signal u tom audio signalu pojavi se zvuk sličan zvuku koji se dobije izgovaranjem riječi 'wah'. Spada u grupu efekata sa vremenski promjenljivim filtrima. 

In [4]:
Fs, x = wavfile.read('acoustic.wav')

In [8]:
def wah_wah(x):
    minf = 500
    maxf = 3000
    Fw = 2000
    delta = Fw/Fs
    Fc = np.arange(minf, maxf, delta)
    while len(Fc) < len(x):
        Fc = np.concatenate((Fc, np.arange(maxf, minf, -delta)))
        Fc = np.concatenate((Fc, np.arange(minf, maxf, delta)))
        
    Fc = np.array(Fc[:len(x)])    
    Q = 2

    y = np.zeros(x.shape)
    for n in np.arange(2, len(x)):
        b, a = iirpeak(2*Fc[n]/Fs, Q)
        y[n] = b[0]*x[n] + b[1]*x[n-1] + b[2]*x[n-2] - a[1]*y[n-1] - a[2]*y[n-2] 

    return y

In [9]:
y = wah_wah(x)
Audio(y, rate=Fs)

### Imlementacija četvrtog efekta - phaser (fejzer)
Efekata jako sličan prethodnom efektu, barem prema implementaciji. Kao i 'wah-wah' efekat pripada grupi efekata sa vremenski promjenljivim filtrima, a razlika je u tome što se kod 'wah-wah' efekta koristi filtar propusnik opsega, dok se kod fejzera koristi filtar nepropusnik opsega odnosno konkretno tzv. 'notch' filtar. 

In [10]:
Fs, x = wavfile.read('acoustic.wav')

In [11]:
def phaser(x):
    minf = 500
    maxf = 3000
    Fw = 2000
    delta = Fw/Fs
    Fc = np.arange(minf, maxf, delta)
    while len(Fc) < len(x):
        Fc = np.concatenate((Fc, np.arange(maxf, minf, -delta)))
        Fc = np.concatenate((Fc, np.arange(minf, maxf, delta)))
        
    Fc = np.array(Fc[:len(x)])

    y = np.zeros(x.shape)

    for n in np.arange(2, len(x)):
        b, a = iirnotch(2*Fc[n]/Fs, 1)
        y[n] = b[0]*x[n] + b[1]*x[n-1] + b[2]*x[n-2] - a[1]*y[n-1] - a[2]*y[n-2]

    x = y
    y = np.zeros(x.shape)
    for n in np.arange(2, len(x)):
        b, a = iirnotch(2*Fc[n]/Fs, 1)
        y[n] = b[0]*x[n] + b[1]*x[n-1] + b[2]*x[n-2] - a[1]*y[n-1] - a[2]*y[n-2]

    return y

In [12]:
y = phaser(x)
Audio(y, rate=Fs)